In [1]:
using BenchmarkTools
using AutoGrad

# Initialization

In [2]:
function init_weights(;n_in=1, n_hidden=100, n_out=1)
    W1 = randn(n_hidden, n_in) # for left multiply W1*x
    b1 = zeros(n_hidden)
    W2 = randn(n_out, n_hidden)
    b2 = zeros(n_out)
    params = [W1, b1, W2, b2]
    return params
end

params = init_weights()
sizes = map(size, params)

4-element Array{Tuple{Int64,Vararg{Int64,N} where N},1}:
 (100, 1)
 (100,)  
 (1, 100)
 (1,)    

In [3]:
x = reshape([-10:0.001:10;],1,:) # input to NN

1×20001 Array{Float64,2}:
 -10.0  -9.999  -9.998  -9.997  -9.996  …  9.996  9.997  9.998  9.999  10.0

# Standard version

In [4]:
function predict(params, x; act=tanh)   
    W1, b1, W2, b2 = params
    a = act.(W1*x .+ b1)
    y = W2*a .+ b2
    return y
end

predict (generic function with 1 method)

In [5]:
@btime predict(params, x)

  55.232 ms (58 allocations: 30.83 MiB)


1×20001 Array{Float64,2}:
 -6.18516  -6.18509  -6.18502  …  6.18495  6.18502  6.18509  6.18516

# Broadcast on x

Treat `predict` as a scalar function and broadcast on x. Might be useful for Autograd.jl.

Wrap the function first. 

In [6]:
predict_wrap = x -> predict(params, x)[1] # assume x is scalar

(::#6) (generic function with 1 method)

**It is 10 times slower!**

In [7]:
@btime predict_wrap.(x[:])'

  386.379 ms (1660118 allocations: 104.38 MiB)


1×20001 RowVector{Float64,Array{Float64,1}}:
 -6.18516  -6.18509  -6.18502  …  6.18495  6.18502  6.18509  6.18516

# Gradient

## Broadcast

In [8]:
predict_grad_v1 = grad(predict_wrap)

(::gradfun) (generic function with 1 method)

In [9]:
@btime predict_grad_v1.(x)

  2.049 s (6960382 allocations: 448.17 MiB)


1×20001 Array{Float64,2}:
 0.0702382  0.0702499  0.0702616  …  0.0702616  0.0702499  0.0702382

## Diagonal of jacobian

In [10]:
sum_predict(x) = sum(predict(params, x))
predict_grad_v2 = grad(sum_predict)

(::gradfun) (generic function with 1 method)

In [11]:
@btime predict_grad_v2(x)

  116.758 ms (324 allocations: 107.60 MiB)


1×20001 Array{Float64,2}:
 0.0702382  0.0702499  0.0702616  …  0.0702616  0.0702499  0.0702382

# Flatten parameters

Flatten all weights into an 1D vector. Useful for Optim.jl.

In [12]:
params_flat = collect(Iterators.flatten(params))
size(params_flat)

(301,)

In [13]:
function predict_flat(params_flat, x; act=tanh)
    # unflatten
    params = []
    i1 = 1
    for s in sizes # sizes is defined outside of the function
        l = reduce(*, s) # size -> length
        i2 = i1+l
        p = reshape(params_flat[i1:i2-1], s)
        push!(params, p)
        i1 = i2
    end 
    
    # normal NN calculation
    
    W1, b1, W2, b2 = params
    
    a = act.(W1*x .+ b1)
    y = W2*a .+ b2
    return y
end

predict_flat (generic function with 1 method)

**Doesn't seem to affect performance**

In [14]:
@btime predict_flat(params_flat, x)

  51.501 ms (88 allocations: 30.83 MiB)


1×20001 Array{Float64,2}:
 -6.18516  -6.18509  -6.18502  …  6.18495  6.18502  6.18509  6.18516